## Ranking

In [2906]:
top_k =16
passage_weights =[1.8,1.6,1.1,1.4,1.4,1,1,1,1,1,1,1,1,1,1,1,1,1,0.9,0.9]

use_print = False
answer_doc =f"./result2/base_answer1.json"  # 답지 파일
predict_doc =f"./result2/result_model4_v1.json"  # 생성하고자 하는 예측 파일 명
predict_score_doc='./result2/score_model4_v1.json' # 생성하고자 하는 예측 파일 점수 파일 명 
output_dir= "/opt/ml/code/output_v2/outputs4" # model inference 시 생성된 output 폴더 
alpha = 1
added = 0

In [2907]:
import json
import numpy as np
from konlpy.tag import Mecab


passage_scores = []
mecab = Mecab()

#get passsage scores
for i in range(top_k):
    with open(f"./passage_score_elastic/passage_scores_e{i+1}.json") as f:
        scores = json.load(f)
        passage_scores.append(scores)

# get n_best_files        
nbest_predictions_list = []
    
for i in range(top_k):
    with open(f"{output_dir}/test_dataset_pk{i+1}/nbest_predictions.json") as f:
        nbest = json.load(f)   
        nbest_predictions_list.append(nbest)
    
keys = nbest_predictions_list[0].keys()
   

In [2908]:
import re
wrong_word=[
            '한때',
            '.',
            '"',
            "최소한",
            "틀림없이",
            "설마",
            "이미",
            "무조건",
            "왜냐하면",
            "이따금",
            "간혹",
            "이미",
            "틀림없이",
            "약간",
            "적어도",
            "가끔",
            "다소",
            "필수",
            "어쩔",
            "대략",
            "한때",
            "설마",
            "괜찮",
            "최소",
            "만약",
            "1811)",
            "때론",
            "노화욱)",
            "절대",
            "기븐",
            "언젠가",
            "정상",
            "만일",
            "가끔 (대략",
            "심지어",
            "비록",
            "않고",
            "않",
            "너무나",
            "만을",
            "감히",
            "반드시",
            "대신들의"
]

invalid_tags = ['MAG', 
                'MAJ', # 부사
                'MAG+XSV+EP+EC'
               ]

invalid_tags_end = [
                'JX'
               ]

def is_proper(word):
    word = word.strip()
    tagged = mecab.pos(word)
    last_word = word.split(" ")
    
    if len(word)>20:
        return False
    if len(tagged) == 1:
        _, tag = tagged[0]
        if tag in invalid_tags:
            return False
    else:
        _, tag = tagged[-1]
        if tag in invalid_tags_end:
            return False
    if last_word[-1] in wrong_word:
        return False 
    if word.endswith('()'):
        return False
    return True

result_dict = {}
result_dict2 = {}

s_max = 0
e_max = 0
s_min = 100
e_min = 100
#11.818488121032715 -10.83463191986084 11.384510040283203 -11.094008445739746
for key in keys:
    scores = np.zeros(top_k)
    texts = []
    p_indices = []
    max_score   = 1 #per passage
    min_score = 100
    for j in range(top_k):
        if passage_scores[j][key] > max_score:
            max_score = passage_scores[j][key]
        if passage_scores[j][key] < min_score:
            min_score = passage_scores[j][key]
    
    for i in range(top_k):
        
        proper_index = 0
        no_proper = True 
        for index,p in enumerate(nbest_predictions_list[i][key]):
            test_text = nbest_predictions_list[i][key][index]['text']
            if is_proper(test_text):
                proper_index = index
                p_indices.append(index)
                texts.append(test_text)
                no_proper = False
                break
                
        # 적당한 것이 하나도 없는 경우    
        if no_proper:
            p_indices.append(-1)
            texts.append("empty")
            scores[i] = -100
        else:
            p_score = passage_scores[i][key]/max_score
            #p_score = ( passage_scores[i][key] - min_score ) / (max_score-min_score) #위에 것이 더 좋다, 
            
            if s_max < nbest_predictions_list[i][key][proper_index]['start_logit'] :
                s_max = nbest_predictions_list[i][key][proper_index]['start_logit'] 
                
            if s_min > nbest_predictions_list[i][key][proper_index]['start_logit'] :
                s_min = nbest_predictions_list[i][key][proper_index]['start_logit'] 
                
            if e_max < nbest_predictions_list[i][key][proper_index]['end_logit']:
                e_max = nbest_predictions_list[i][key][proper_index]['end_logit']
            if e_min > nbest_predictions_list[i][key][proper_index]['end_logit']:
                e_min = nbest_predictions_list[i][key][proper_index]['end_logit']
            
          
            logit_score = alpha*(added+nbest_predictions_list[i][key][proper_index]['start_logit'])+(added+nbest_predictions_list[i][key][proper_index]['end_logit'])
            scores[i]  = logit_score*p_score*passage_weights[i]
    
    
    sorted_indice = scores.argsort()        
    
    result_dict[key] = texts[sorted_indice[-1]]
    result_dict2[key] = scores[sorted_indice[-1]]
    if use_print:
        print(key,texts[sorted_indice[-1]])

        print(texts[sorted_indice[-2]],texts[sorted_indice[-3]],texts[sorted_indice[-4]],
            round(scores[sorted_indice[-1]],4) ,
            round(scores[sorted_indice[-2]],4) ,
            round(scores[sorted_indice[-3]],4), 
            round(scores[sorted_indice[-4]],4),
            sorted_indice[-1],
            sorted_indice[-2],
            sorted_indice[-3],
            sorted_indice[-4]
             )

   
with open(predict_doc, "w") as writer:
    writer.write(json.dumps(result_dict, indent=4, ensure_ascii=False) + "\n")
with open(predict_score_doc, "w") as writer:
    writer.write(json.dumps(result_dict2, indent=4, ensure_ascii=False) + "\n")
print('done')

done


## 답지 체크

In [2909]:
with open(answer_doc) as f1:
    answer = json.load(f1)
    
with open(predict_doc) as f2:
    prediction = json.load(f2)

cnt = 0
for key in answer.keys():
    if answer[key] != prediction[key]:
        if True:
            print(key,answer[key],":",prediction[key])
        cnt +=1
        
print("count:",cnt)

mrc-1-001113 냉전 : 플레이오프
mrc-1-000993 유두 : 아래턱
mrc-0-001606 그로인 볼트 : 그로인볼트로
mrc-0-001206 1970년 : 2007년
mrc-1-000418 일등바위 : 228m
mrc-0-004646 당역 : 건늠선
mrc-0-001058 룸비니 : 투리야(Turiya)의 세계
mrc-1-001555 구마가야 고등학교 : 소토쿠 고등학교(히로시마현)
mrc-0-000504 고려초기 : 삼국시대
mrc-0-003763 조성발원문(造成發願文) : 광배(光背)의 뒷면
mrc-0-003177 연금 : "연금"
mrc-1-001523 니 기븐 : 기욤
mrc-0-002453 요미우리 OB : 노무라
mrc-0-004430 십자가 밑 : '흙 냄새가 싫다.'
mrc-0-000432 한중호조사 : 이르쿠츠크파 고려공산당 상하이 지부
mrc-0-000540 빌바인 : 오라 소드
mrc-0-001235 화석 : 하늘
mrc-1-000813 마리아나 제도에서 벗어나자 : 마리아나 제도에서 벗어나
mrc-0-001346 손권 : 호해
mrc-0-003364 아우구스투스 시대(기원전 27년 ~ 기원후 14년) : 기원후 1세기
mrc-1-001239 일본어 : 쉴드 빔
mrc-0-003110 아즈마비토 가문 : 난씨
mrc-0-000755 즈믄둥이 : '베이비복스 리브'
mrc-1-001457 임진왜란 : 도요토미 히데요시
mrc-1-000521 평안북도 용천군 : 응봉산 중턱
mrc-0-001699 MKS 단위계 : SI 단위계
mrc-1-000116 제우스 : 법관
mrc-0-002210 통인 주기 : 란다
mrc-0-000986 제프 허스트 : 손흥민
mrc-0-002771 궤도 : 카시니 탐사선
mrc-0-002975 히즈루국 : 고구려
mrc-0-001021 첫 번째 경우 : 뇌
mrc-0-005465 쿠파(クッパ, Bowser) : 쿠파
mrc-0-001379 타히티 축구 국가대표팀 : 나이지리아
mrc-0-001445 

## Ensemble

In [2910]:
import numpy as np
score_list= []
model_counts= 6

for i in range(model_counts):
    with open(f"./result2/score_v10_15k_v{i+1}.json") as f:
        scores = json.load(f)
        score_list.append(scores)

keys = score_list[0].keys()
dict1 = {}
weight_list=[1.5,1.5,1,1,1,1]
for i,score in enumerate(score_list):
    for key in keys:
        temp_list =[]
        if key in dict1:
            temp_list = dict1[key]
        else:
            dict1[key]  = temp_list
        temp_list.append(score[key]*weight_list[i])
        
text_list= []       
for i in range(model_counts):
    with open(f"./result2/result_v10_15k_v{i+1}.json") as f:
        texts = json.load(f)
        text_list.append(texts)
        
final_result={}
for key in dict1.keys():
    temp_list= dict1[key]
    print(temp_list)
    temp_list2 = np.array(temp_list)
    indice  = temp_list2.argsort()
    final_result[key]= text_list[indice[-1]][key]
    print(key,indice[-1],text_list[indice[-1]][key])

with open('./result2/merged1.json', "w") as writer:
    writer.write(json.dumps(final_result, indent=4, ensure_ascii=False) + "\n")
    
print("done")   

[56.965046882629395, 797.8273867881314, 585.1281910360591, 37.97669792175293, 1.6953887385129929, 60.119031047821046]
mrc-1-000653 1 베텔게우스(Betelgeuse)
[42.015349817276004, 538.9702631336593, 415.04892666490605, 28.010233211517335, 1.7348923069238662, 58.43775596618653]
mrc-1-001113 1 냉전
[74.23411531448365, 1159.36616339929, 817.5914954300764, 49.489410209655766, 1.8259629124403, 66.99497623443604]
mrc-0-002191 1 대통령인 빌헬름 미클라스
[72.86222591400147, 1128.5304264953609, 797.9067789991896, 48.57481727600098, 2.0899033308029176, 67.11992340087892]
mrc-0-003951 1 뉴질랜드
[74.22158060073852, 1159.4798990582206, 817.6393381483649, 49.48105373382568, 2.0830243706703184, 65.51060733795167]
mrc-1-001272 1 프랑스군
[30.7681102012456, 331.903107988479, 272.74577201645974, 23.076082650934197, 1.8228722752127018, 20.582210592387476]
mrc-1-000993 1 유두
[58.868849229812625, 834.35679840884, 608.8397020403141, 39.24589948654175, 1.7038580065965652, 56.13403406143189]
mrc-0-005021 1 구체제
[32.3091310058564, 432.2421